In [1]:
#%pip install -U spaCy
#%pip install keras
#%python -m spacy download en_core_web_sm
#%pip install -U jax jaxlib
#%pip install clean-text
#%pip install --upgrade ipykernel
#%pip install tensorflow


#da eseguire al primo utilizzo

In [2]:
import string
import spacy
import json
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.it.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
from sklearn.svm import SVC
from sklearn import metrics
from sklearn import svm
from sklearn import neural_network
from sklearn import ensemble
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from pandas import read_csv
import nltk
import random
from sklearn.model_selection import KFold
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.utils import class_weight
import sklearn.metrics
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.svm import SVC

#init spaCy
punctuations = string.punctuation
nlp = spacy.load("en_core_web_lg")
stop_words = spacy.lang.en.stop_words.STOP_WORDS
parser = English()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    translator = str.maketrans("", "", string.punctuation)
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation.lower()
    #return text.strip().lower()

# Tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens



In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\perna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df = read_csv('../Dataset/final_dataset_sexist.csv')

X = df['text']
y = df['label_sexist']
print(y.value_counts())
df.set_index('ID')


not sexist    15146
sexist         4854
Name: label_sexist, dtype: int64


,text,label_sexist
ID,,
0,"In Nigeria, if you rape a woman, the men rape ...",not sexist
1,"Then, she's a keeper. 😉",not sexist
2,This is like the Metallica video where the poo...,not sexist
3,woman?,not sexist
4,I bet she wished she had a gun,not sexist
...,...,...
19995,girls really get fucked almost every weekend ?,not sexist
19996,The hatred for moslems are Reasonable and Just...,not sexist
19997,Now this is a woman who gets it. 👆,not sexist


In [5]:
vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [6]:
#Linear Support Vector Machine classifier
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
accuracy_label = []
precision_label = []
recall_label = []
f1_label = []

class_labels = np.unique(y)
classifier = svm.LinearSVC(max_iter = 10000, class_weight= {"not sexist": 0.2, "sexist": 1})
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier(hidden_layer_sizes=(512,),verbose=True,max_iter=10)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced')

# Create the pipeline

pipe = Pipeline([("cleaner", predictors()),
('vectorizer', vector),
('classifier', classifier)])

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X):
    # Divisione dei dati in fold di addestramento e di test
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Addestramento del modello
    pipe.fit(X_train, y_train)

    # Valutazione delle performance del modello sul test set
    predicted = pipe.predict(X_test)

    # Calcolo delle metriche di valutazione per il fold corrente
    accuracy = accuracy_score(y_test, predicted)
    precision = precision_score(y_test, predicted, average='weighted')
    recall = recall_score(y_test, predicted, average='weighted')
    f1 = f1_score(y_test, predicted, average='weighted')

    # Aggiunta delle metriche del fold corrente alle liste
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calcolo delle metriche per ogni singola classe
    precision_label = precision_score(y_test, predicted, average=None, labels=class_labels)
    recall_label = recall_score(y_test, predicted, average=None, labels=class_labels)
    f1_label = f1_score(y_test, predicted, average=None, labels=class_labels)

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
#y_test = y_test.reset_index(drop=True)

In [7]:
mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)
mean_f1 = np.mean(f1_scores)

# Stampa delle metriche
print("---------------------------------")
print("Metriche finali:")
print(f"Accuratezza media: {mean_accuracy:.4f}")
print(f"Precisione media: {mean_precision:.4f}")
print(f"Richiamo medio: {mean_recall:.4f}")
print(f"F1-score medio: {mean_f1:.4f}")
print("---------------------------------\n")

# Stampa delle metriche per ogni singola classe
print("---------------------------------")
print("Metriche per ogni singola classe:")
for i, label in enumerate(class_labels):
    print(f"Classe '{label}':")
    print(f"Precisione: {precision_label[i]:.4f}")
    print(f"Richiamo: {recall_label[i]:.4f}")
    print(f"F1-score: {f1_label[i]:.4f}")
    print()
print("---------------------------------")

---------------------------------
Metriche finali:
Accuratezza media: 0.7848
Precisione media: 0.7971
Richiamo medio: 0.7848
F1-score medio: 0.7897
---------------------------------

---------------------------------
Metriche per ogni singola classe:
Classe 'not sexist':
Precisione: 0.8838
Richiamo: 0.8297
F1-score: 0.8559

Classe 'sexist':
Precisione: 0.5472
Richiamo: 0.6534
F1-score: 0.5956

---------------------------------


In [8]:
df1 = read_csv('../Dataset/final_dataset_category.csv')
X1 = df1['text']
y1 = df1['label_category']
df1.set_index('ID')
print(y1.value_counts())


2. derogation                               2271
3. animosity                                1665
4. prejudiced discussions                    475
1. threats, plans to harm and incitement     443
Name: label_category, dtype: int64


In [9]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vector1 = TfidfVectorizer(tokenizer = tokenizer.tokenize, ngram_range=(1,2))

In [18]:
from sklearn.model_selection import GridSearchCV

#Linear Support Vector Machine classifier
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
accuracy_label = []
precision_label = []
recall_label = []
f1_label = []

class_labels = np.unique(y1)
#classifier = svm.LinearSVC(max_iter = 10000, class_weight= "balanced")
# Multi-layer Perceptron classifier
classifier = neural_network.MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(15,),verbose=True)
# Random Foreset
#classifier = ensemble.RandomForestClassifier(class_weight='balanced', verbose = True)



# Create the pipeline
pipe1 = Pipeline([("cleaner", predictors()),
('vectorizer', vector1),
('classifier', classifier)])

kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X1):
    # Divisione dei dati in fold di addestramento e di test
    X1_train, X1_test = X1[train_index], X1[test_index]
    y1_train, y1_test = y1[train_index], y1[test_index]
    # Addestramento del modello
    pipe1.fit(X1_train, y1_train)

    # Valutazione delle performance del modello sul test set
    predicted = pipe1.predict(X1_test)

    # Calcolo delle metriche di valutazione per il fold corrente
    accuracy = accuracy_score(y1_test, predicted)
    precision = precision_score(y1_test, predicted, average='weighted')
    recall = recall_score(y1_test, predicted, average='weighted')
    f1 = f1_score(y1_test, predicted, average='weighted')

    # Aggiunta delle metriche del fold corrente alle liste
    accuracy_scores.append(accuracy)
    precision_scores.append(precision)
    recall_scores.append(recall)
    f1_scores.append(f1)

    # Calcolo delle metriche per ogni singola classe
    precision_label = precision_score(y1_test, predicted, average=None, labels=class_labels)
    recall_label = recall_score(y1_test, predicted, average=None, labels=class_labels)
    f1_label = f1_score(y1_test, predicted, average=None, labels=class_labels)


Iteration 1, loss = 1.24261233
Iteration 2, loss = 1.20686011
Iteration 3, loss = 1.15180400
Iteration 4, loss = 1.06744406
Iteration 5, loss = 0.96627410
Iteration 6, loss = 0.85636233
Iteration 7, loss = 0.74464238
Iteration 8, loss = 0.63753790
Iteration 9, loss = 0.53793671
Iteration 10, loss = 0.44965583
Iteration 11, loss = 0.37365150
Iteration 12, loss = 0.31002639
Iteration 13, loss = 0.25756160
Iteration 14, loss = 0.21515728
Iteration 15, loss = 0.18115161
Iteration 16, loss = 0.15352989
Iteration 17, loss = 0.13136486
Iteration 18, loss = 0.11321679
Iteration 19, loss = 0.09847327
Iteration 20, loss = 0.08622505
Iteration 21, loss = 0.07603817
Iteration 22, loss = 0.06751446
Iteration 23, loss = 0.06028702
Iteration 24, loss = 0.05416799
Iteration 25, loss = 0.04889772
Iteration 26, loss = 0.04434087
Iteration 27, loss = 0.04037237
Iteration 28, loss = 0.03691172
Iteration 29, loss = 0.03388109
Iteration 30, loss = 0.03121121
Iteration 31, loss = 0.02884749
Iteration 32, los

In [19]:
mean_accuracy = np.mean(accuracy_scores)
mean_precision = np.mean(precision_scores)
mean_recall = np.mean(recall_scores)
mean_f1 = np.mean(f1_scores)

# Stampa delle metriche
print("---------------------------------")
print("Metriche finali:")
print(f"Accuratezza media: {mean_accuracy:.4f}")
print(f"Precisione media: {mean_precision:.4f}")
print(f"Richiamo medio: {mean_recall:.4f}")
print(f"F1-score medio: {mean_f1:.4f}")
print("---------------------------------\n")

# Stampa delle metriche per ogni singola classe
print("---------------------------------")
print("Metriche per ogni singola classe:")
for i, label in enumerate(class_labels):
    print(f"Classe '{label}':")
    print(f"Precisione: {precision_label[i]:.4f}")
    print(f"Richiamo: {recall_label[i]:.4f}")
    print(f"F1-score: {f1_label[i]:.4f}")
    print()
print("---------------------------------")

---------------------------------
Metriche finali:
Accuratezza media: 0.5295
Precisione media: 0.5459
Richiamo medio: 0.5295
F1-score medio: 0.5041
---------------------------------

---------------------------------
Metriche per ogni singola classe:
Classe '1. threats, plans to harm and incitement':
Precisione: 0.8636
Richiamo: 0.4222
F1-score: 0.5672

Classe '2. derogation':
Precisione: 0.5229
Richiamo: 0.7512
F1-score: 0.6166

Classe '3. animosity':
Precisione: 0.5278
Richiamo: 0.4471
F1-score: 0.4841

Classe '4. prejudiced discussions':
Precisione: 0.6923
Richiamo: 0.1579
F1-score: 0.2571

---------------------------------


In [ ]:
single_instance = ["I can smell her dying from here."]
prediction = pipe.predict(single_instance)

if (prediction == 'sexist'):
    prediction = pipe1.predict(single_instance)

print(f"Classe: {prediction}")

Classe: ['1. threats, plans to harm and incitement']
